# BorutaPy Feature_Selection


- ## Boruta의 알고리즘 단계별로 살펴보기
- 새로운 변수 생성
    - 원본 데이터의 독립변수를 복사하여 생성된 변수들, 이 변수는 원본 변수와 같은 데이터 분포를 가지지만 예측에는 아무런 의미가 없도록 -> 무작위로 섞는 것 (5개 이상을 최대한 섀도우 변수로 만든다. (통계를 얻기 위해)
    
- 원본 데이터와 섀도우 변수를 결합
    - 새로운 데이터 셋이 학습 데이터를 만든다.
    - 원본 + 섀도우 변수가 같이 들어간 학습 데이터
    
- RandomForest 모델 학습과 Z-Score를 계산
    - 랜덤 포레스트로 모델 학습
    - 각 특성의 중요도를 랜덤 포레스트의 트리에서 계산된 정확도 손실을 바탕으로 측정
    - 정규화해서 Z-score , 이 특성이 결국 랜덤포레스트 모델에서 얼마나 중요한지를 나타내는 것, 이 값이 높을 수록 타겟 변수와의 연관성이 높다고 간주하는 것
    - **Z-score = ((각 트리에 대한 손실 - 전체 트리의 정확도 손실의 평균) / 정확도 손실의 표준편차)**
  
- 섀도우 변수 중 가장 높은 Z-score (MSZA, Max Z-score among shadow attributes)
    - 각 섀도우 변수를 Z-score 가장 높은 MSZA
    - 섀도우 변수들 중에서 타겟 변수와 가장 높은 상관 관계를 가지는 변수가 나올 것
    - MSZA는 원본 변수의 Z-score를 평가할 기준이 된다. 이 값보다 높은 Z-score를 가진 원본 변수가 중요한 변수로 판단
- 중요한 변수, 중요하지 않은 변수 태그 될 때까지 계속해서 반복횟수 max_iter 과정 수행

## Boruta에서 랜덤하게 섞는 이유?
- 섀도우 변수를 랜덤하게 섞는 건 원본 특성을 무작위로 섞은 것
- 이 특성은 타겟 변수와 연관성이 거의 없어진다. 섀도우 특성의 중요도가 높다면 이건 우연에 의한 가능성이 크다.
- 랜덤하게 다 섞어서 나온 값인데, 이게 특성 중요도가 높다고 나오면? 우연일 가능성이 높음
- 최소한 우연보다는 score가 높아야지 특성의 중요도가 높은 것 아닌가? -> 원본 변수와 비교
- 원본 특성의 중요도가 무작위 특성(섀도우 특성)보다 통계적으로 유의미하게 높은지를 검정
- 섀도우 특성을 활용해서 통계적으로 유의미한 특성만 남기고 나머지 제거

In [1]:
## 의사결정나무(Decision Tree)
## 랜덤포레스트
## 부트스트랩 샘플링
## 트리에서 불순도 감소 계산
## 특성의 중요도를 어떤식으로 계산하는가?

In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

np.random.seed(0)
X = np.random.rand(100,3)
y = (X[:,0] + X[:,1]*0.5 + X[:,2] * 0.2>0.5).astype(int)


model = RandomForestClassifier(n_estimators=100, random_state=111)


In [3]:
model.fit(X,y)

feature_importances = model.feature_importances_

In [4]:
for i , importance in enumerate(feature_importances):
    print(f'Feature {i+1} Importance:{importance:.4f}')

Feature 1 Importance:0.5310
Feature 2 Importance:0.2967
Feature 3 Importance:0.1723


## 퍼뮤테이션 테스트
- Permutation Test
    - 데이터를 무작위로 섞는 방식을 통해 두 그룹 간의 차이가 실제로 유의미한지 여부를 검정하는 비모수적 통계기법
    데이터의 정규성을 가정하지 않고, 데이터에서 직접 추출된 분포를 사용해서 검정을 수행하기 때문에 어떤 분포든 적용할 수 있다.
- 가설 검정:
    - 귀무가설 : 두 그룹 간에 차이가 없다. 관찰된 차이는 우연히 발생한 것이다.
    - 대립가설 : 두 그룹 간에 실제로 차이가 있다.
    - 수 천, 수 만번 계산해서 샘플에 대한 통계량을 가지고 차이를 보는 것

## Multicollinearity 다중공선성
- x1, x2 둘 다 매우 높은 상관관계를 가지고 있다.
- 두 개 이상의 독립변수가 서로 강한 상관관계를 가지고 있을 때 발생하는 다중공선성
- 문제는?
    - 모델 해석의 어려움 - 어떤 게 영향을 주는 변수인지 모름. 동일한 정보를 주니깐 두 변수 중 어떤 것이 실제로 종속변수에 중요한 지를 구분하기 어려움
    - 회귀계수가 불안정해짐. - 상관 관계가 높으면 데이터의 작은 변화가 회귀계수를 큰 폭으로 변하게 만들 수 있어서 모델의 해석이 어렵고 신회도가 낮아질 수 있음
    - 잘못된 평가로 이어질 수 있음 - x1, x2 둘의 영향으로 다중 공선성이 만들어 지는데, 둘 중 하나만 지우면 다중공선성은 안정되겠지만, 실제로 어떤게 중요한 변수인지는 실제 도메인 등을 고려해서 작업해야 하는 부분이 있다.
- 다중 공선성 측정
    - 분산팽창요인 (VIF, Variance Inflation Factor) 각 독립변수가 나머지 독립변수들로 얼마나 잘 설명되는지 VIF가 10 이상이면 다중공선성이 있다고 판단
    - 상관행렬: 상관계수를 구해서 확인 가능

## Boruta에서 왜 이부분을 같이 확인할까?
- 랜덤포레스트가 특성 간의 상관관계가 강하면 일부 특성이 중요하지 않다고 판단할 수 있따. (잘못평가할 확률)
- Boruta에서 사용하면 랜덤포레스트가 특성 중요도를 활용해서 중요 특성을 선택한다.
    - 잘못된 판단을 한쪽 변수가 중요하지 않다고 생각하여 확인할 수 있다.
    - 다중공선성으로 인해 랜덤포레스트가 특정 특성을 과소평가할 위험이 있고, boruta가 이를 보완해준다?
- 따라서 boruta 사용 전에 피처들간의 상관관계를 꼭 살펴보고, 피처가 너무 많은 경우는 차원 축소 등의 방법을 사용하는 것을 권장

- boruta는 시계열 데이터에서는 권장 X